In [ ]:
!pip install numpy pandas sklearn minisom

In [ ]:
!pip install minisom

  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.2-py3-none-any.whl size=10650 sha256=e09cc99abf423f58f8be70373396ad24d325ef602032f2486bc454c1d8e21048
  Stored in directory: /root/.cache/pip/wheels/b4/f6/8a/9daf8831901c3e3805775633404248f10663d1c80b7e5a1314
Successfully built minisom


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from minisom import MiniSom
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# df = pd.read_csv('/content/combined Dataset.csv')

In [ ]:
# Load the dataset
url = "https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/combined%20Dataset.csv"
df = pd.read_csv(url)
df.shape

In [ ]:
# Assuming the dataset has the necessary columns based on the paper's description
# Features: 'etpSrcIP', 'etpSrcP', 'etpDstP', 'etpProtocol', 'totalPacket'
# Label: 'isAttack'
X = df[['etpSrcIP', 'etpSrcP', 'etpDstP', 'etpProtocol', 'totalPacket']].values


In [ ]:
y = df['label'].values

In [ ]:
# Data preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Splitting dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


In [ ]:
# Training SOM
som_size = int(np.sqrt(5*np.sqrt(X_train.shape[0])))  # heuristic for SOM size
som = MiniSom(som_size, som_size, X_train.shape[1], sigma=1.0, learning_rate=0.5)
som.train_random(X_train, 1000)  # training with 1000 iterations


In [ ]:
# Mapping training data to SOM and creating a new dataset with BMU positions
X_train_bmu = np.array([som.winner(x) for x in X_train])
X_test_bmu = np.array([som.winner(x) for x in X_test])


In [ ]:
# SOM + k-NN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_bmu, y_train)
score = knn.score(X_test_bmu, y_test)


In [ ]:
print(f'Accuracy of SOM + k-NN: {score*100:.2f}%')

Accuracy of SOM + k-NN: 100.00%
